In [1]:
import tensorflow as tf
import pickle
import keras
from keras.models import Model, load_model, clone_model
from keras.utils import multi_gpu_model
from keras.layers import *
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.losses import binary_crossentropy
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import sys
import random
sys.path.append('src')  # Fix for jupyter
import src.emulator as emulator
import src.emulator_utils as emulator_utils
import src.emulator_vis as emulator_vis
%matplotlib inline
from IPython.display import clear_output
import time
import multiprocessing
from multiprocessing import Event, Queue, Pipe
from multiprocessing import Process as Thread
import os
import logging
from mcts import *
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/adryw/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/adryw/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


# Logging

In [2]:
logging.basicConfig(filename='logging.log', level=logging.INFO, format='%(asctime)s %(message)s', filemode='w')

# Model Params

In [3]:
INPUT_SIZE = (None, None, 5) # Map size fixed to 16x16 (2 to 3 players)
#INPUT_SIZE = (13, 13, 5) # Map size fixed to 16x16 (2 to 3 players)
N_ACTIONS = 4
gpus = 1

# Define the Layers Blocks

In [4]:
filters = 64

# Convolutional Block
def conv_block(in_layer, name, filters=filters, kernel_size=(3,3), bn=True, relu=True):
    l = Conv2D(filters, kernel_size, use_bias = False, 
               padding='same', name = name, kernel_regularizer=l2(1e-4))(in_layer)
    if bn:
        l = BatchNormalization(axis=3, name = name + '_bn')(l)
    if relu:
        #l = Activation('relu', name = name + '_relu')(l)
        l = LeakyReLU(name = name + '_lkrelu')(l)

    return l

# Residual Block
def residual_conv(in_layer, idx, filters=filters, kernel_size=(3,3), bn=True, relu=True):
    name = 'res_' + str(idx)
    # Full conv block of pre-defined shape
    l = conv_block(in_layer, name + '_conv1', filters, kernel_size=(3,3), bn=True, relu=True)
    # Second block with skip connection
    l = Conv2D(filters, kernel_size, use_bias = False, padding='same', 
               name = name + '_conv2', kernel_regularizer=l2(1e-4))(l)
    if bn:
        l = BatchNormalization(axis=3, name = name + '_conv2_bn')(l)
    
    l = Concatenate()([in_layer, l]) # Skip conn.
    #l = Add()([in_layer, l]) # Skip conn.
    
    if relu:
        #l = Activation('relu', name = name + '_relu')(l)
        l = LeakyReLU(name = name + '_lkrelu')(l)
        
    return l

def value_head(in_layer):
    l = conv_block(in_layer, 'value_conv', filters=64)
    l = Conv2D(name='value_head', filters=1, kernel_size=(1,1), activation='tanh', 
               kernel_regularizer=l2(1e-4), use_bias=False)(l)

    l = Lambda(lambda x : x / tf.math.square(tf.cast(tf.shape(x)[1], dtype=tf.float32)), name='value_rescaled')(l)
    l = Lambda(lambda x : tf.reduce_sum(x, axis=[1, 2]), name='value_h')(l)
    l = Lambda(lambda x : x * 2, name='value_rescale')(l)
    l = Lambda(lambda x : tf.clip_by_value(x, -1., 1.), name='value')(l)
    return l

def policy_head(in_layer):
    l = conv_block(in_layer, 'policy_conv', filters=64)
    l = Conv2D(name='policy_head', filters=4, kernel_size=(1,1), 
               kernel_regularizer=l2(1e-4), activation='sigmoid')(l)
    
    l = Lambda(lambda x : tf.reduce_sum(x, axis=[1, 2]), name='sum_policy')(l)
    l = Activation('softmax', name='policy')(l)
    
    return l

# Model

In [5]:
def create_model():
    def declare_model():
        n_residual = 8

        input_layer = Input(INPUT_SIZE)
        l = conv_block(input_layer, 'conv')
        for i in range(n_residual):
            l = residual_conv(l, idx=i + 1)

        policy = policy_head(l)
        value = value_head(l)

        alphabot = Model(input_layer, [policy, value])
        return alphabot
    
    if gpus > 1:
        with tf.device('/cpu:0'):
            alphabot = declare_model()
        alphabot_multi = multi_gpu_model(alphabot, gpus=gpus)
        return alphabot_multi, alphabot
    
    alphabot = declare_model()
    return alphabot, alphabot

In [6]:
def policy_rot90(policy, k = 1):
    k = k % 4
    policy = np.array(policy)
    for i in range(k):
        policy = policy[..., [1, 2, 3, 0]]
    
    return policy

def policy_flip(policy, vert=False):
    policy = np.array(policy)
    if vert:
        return policy[..., [0, 3, 2, 1]]
    
    return policy[..., [2, 1, 0, 3]]

def state_flip(state, vert=False):
    state = np.array(state)
    
    if vert:
        return state[:, ::-1]
    return state[:, :, ::-1]

def apply_simmetries(train_steps):
    # 90;180:270 degrees rotations
    # 0 right, 1 down, 2 left, 3 up
    # Flips
    
    t_s = []
    t_p = []
    value = []
    for step in train_steps:
        t_s.append(step.state)
        t_p.append(step.policy)
        value.append(step.value)
    
    for i in range(1, 4):
        for j in range(0, 2):
            state = np.rot90(t_s, k=i, axes=(1, 2))
            policy = policy_rot90(t_p, k=i)
    
            if j == 0: # Horizontal flip
                state = state_flip(state, vert=False)
                policy = policy_flip(policy, vert=False)
            elif j == 1: # Vertical flip
                state = state_flip(state, vert=True)
                policy = policy_flip(policy, vert=True)

            train_steps.extend([TrainStep(s, v, p) for s, p, v in zip(state, policy, value)])
    
    return train_steps

In [7]:
def manage_predictions():
    t = 0
    
    while not winner_buffer.full():
        indices1, states1 = [], []
        indices2, states2 = [], []
        
        if processable_buffer.qsize() < min(num_threads, num_threads//3): # Wait until a bunch of requests are queued
            continue

        net0, net1 = [], []
        for i in range(processable_buffer.qsize()):
            index, state, net = processable_buffer.get()
            if net == False:
                if state[..., -1].all() == 0:
                    net = 'alphabot'
                else:
                    net = 'alphabot_best'
            else:
                if state[..., -1].all() == 0:
                    net = 'alphabot_best'
                else:
                    net = 'alphabot'
                
            if net == 'alphabot':
                indices1.append(index)
                states1.append(state)
            elif net == 'alphabot_best':
                indices2.append(index)
                states2.append(state)
        
        predictions1, predictions2 = [], []
        if len(states1) > 0:
            states1 = np.array(states1, dtype=np.float32)
            predictions1 = alphabot.predict(states1)
        if len(states2) > 0:
            states2 = np.array(states2, dtype=np.float32)
            predictions2 = alphabot_best.predict(states2)

        if len(predictions1) > 0:
            for i, pred in enumerate(tuple(zip(predictions1[0], predictions1[1]))):
                pipes[indices1[i]].send(dict(zip(alphabot.output_names, pred)))
        
        if len(predictions2) > 0:
            for i, pred in enumerate(tuple(zip(predictions2[0], predictions2[1]))):
                pipes[indices2[i]].send(dict(zip(alphabot.output_names, pred)))
        
        if time.time() - t > 30: # Every 30 secs
            t = time.time()
            logging.info('Finished evaluation %d games' % winner_buffer.qsize())
        
def simulate_games():
    logging.debug('Starting Threads for parallel Games')
    
    parallel_sim(evaluation=False) # Parallel Games
    
    while not history_buffer.full():
        indices, states = [], []
        if processable_buffer.qsize() < min(num_threads, 2): # Wait until a bunch of requests are queued
            continue

        for i in range(processable_buffer.qsize()):
            index, state, _ = processable_buffer.get()
            indices.append(index)
            states.append(state)
            
        states = np.array(states, dtype=np.float32)
        predictions = alphabot.predict(states)
        for i, pred in enumerate(tuple(zip(predictions[0], predictions[1]))):
            pipes[indices[i]].send(dict(zip(alphabot.output_names, pred)))

    logging.info('Finished Simulating %s games', n_games)

In [8]:
def play_eval(reverted=False, pipe=None, process_id=None):
    global alphabot_best
    global alphabot
    
    game = emulator.Game(2)
    mapp = game.reset()
  
    tree_player0 = MCTS()
    tree_player0.alpha = MCTS_eval_alpha
  
    tree_player1 = MCTS()
    tree_player1.alpha = MCTS_eval_alpha

    old_mapp = None
    head = None
    turn = 0
    s = map_to_state(mapp, old_mapp, None, 0)
    old_mapp = copy.deepcopy(mapp)
  
    policies = []    
    while True:
        if turn == 0:
            policy = do_search(MCTS_eval_steps, s, mapp, game, tree_player0, pipe=pipe, process_id=process_id, ask_predict=ask_predict, alphabot=reverted, allow_move=allow_move)
        else:
            policy = do_search(MCTS_eval_steps2, s, mapp, game, tree_player1, pipe=pipe, process_id=process_id, ask_predict=ask_predict, alphabot=reverted, allow_move=allow_move)

        if not use_eval_choice:
            choosen = np.argmax(policy)
        else :
            choosen = np.random.choice(4, p=policy)

        policies.append(np.array(policy))
        mapp, tmp_head = game.step(mapp, s, choosen, turn, mcts=True)

        turn = 1 - turn
        if turn == 0:  # We update the state
            s = map_to_state(mapp, old_mapp, s, 0, head)  # TODO: Map to state
        else:
            head = tmp_head
            s[..., -1] = 1

        if turn == 0:
            old_mapp = np.array(mapp)
        
        if game.game_ended():
            logging.debug('GAME ENDED, %s won, %s <- reverted' % (turn, reverted))
            if not reverted:
                return turn
            else:
                return  1 - turn

In [9]:
def train_model():        
        picked_data = random.sample(complete_history, k=min(BATCH_SIZE, len(complete_history))) 
        
        state = []
        policy = []
        value = []
        for step in picked_data:
            policy.append(step.policy)
            state.append(step.state)
            value.append(step.value)
            
        y = [np.zeros((len(state), 4)), np.zeros((len(state), 1))]
        y[0] = policy
        y[1] = value
        
        losses = alphabot.train_on_batch(np.array(state, dtype=np.float32), y)
        return losses

In [10]:
def training_cycle():
    global alphabot
    global alphabot_best
    global total_improv
    
    logging.info('Starting Training Cycle')
    simulate_games()
    
    # history_buffer contains the games, we store them inside complete history    
    tmp_buffer = []
    for g in range(history_buffer.qsize()):
        tmp_buffer.append(history_buffer.get())
    tmp_buffer = apply_simmetries(tmp_buffer)
    complete_history.extend(tmp_buffer)
    stop_simulation() # We can now stop the simulation (will free the memory)
    
    logging.info('Starting Model Training')
    losses = [0, 0, 0] # For debug purpose
    sum_loss = 0
    cc = 1
    for i in range(t_steps + 1):
        if i % 100 == 0:
            logging.info('Training Interaction: %s losses: %s %s', i, 
                         round(sum_loss / cc, 2), np.round(losses, 2))
        
        losses = train_model()
        sum_loss += losses[0]
        logging.debug('Losses: %s', losses)
        
        cc += 1
        if i % eval_steps == 0 and i > 0:
            cc = 1 # Reset loss counter
            sum_loss = 0
            wins = {'candidate' : 0, 'best' : 0}
            n_c = {0 : 'candidate', 1 : 'best'}
            
            logging.info('Starting self-play evaluation')    
            parallel_sim(evaluation=True) # Start Parallel Games
            manage_predictions()
            for i in range(winner_buffer.qsize()):
                w = winner_buffer.get()
                wins[n_c[w]] += 1 # add a win to the winner
            stop_simulation()
            
            win_ratio = round(wins['candidate'] / eval_games, 2)
            if win_ratio >= win_percent:
                logging.info('Great! Our candidate won %s percent of games', round(win_ratio * 100))
                total_improv += 1
                logging.info('Our bot got better %s times', total_improv)
                alphabot.save('alphabot_best.pickle')
                replace_best()    
            else:
                logging.info('Damn! Our candidate only won %s percent of games', round(win_ratio * 100, 2))         
                logging.info('Cloning to best')
                reload_best()
            
    if len(complete_history) >= k * n_games * 7:
        logging.info('Removing oldest games')
        del complete_history[:n_games * 7] # Delete n oldest games from history

In [11]:
def load_best(best_model):
    global alphabot
    global alphabot_best
    alphabot_best = load_model(best_model, 
                               custom_objects={'softmax_cross_entropy_with_logits' : softmax_cross_entropy_with_logits,
                                          'categorical_weighted' : categorical_weighted})
    alphabot = load_model(best_model, 
                               custom_objects={'softmax_cross_entropy_with_logits' : softmax_cross_entropy_with_logits,
                                          'categorical_weighted' : categorical_weighted})

In [12]:
def reload_best():
    global alphabot
    global alphabot_best
    
    alphabot.set_weights(alphabot_best.get_weights())

In [13]:
def replace_best():
    global alphabot
    global alphabot_best
    
    alphabot_best.set_weights(alphabot.get_weights())

In [14]:
def train(cycles):
    global alphabot_best
    global alphabot
    
    #replace_best()
    
    complete_history = []
    for i in range(cycles):
        logging.info('Training cycle: %s', i)
        training_cycle()

In [15]:
def ask_predict(idi, x, net=None):
    # Adds to queue id and data from process
    processable_buffer.put((idi, x, net))

def sim(process_id, pipe, evaluation=False):
    np.random.seed()
    random.seed()
    
    if evaluation:
        while True:
            reverted = np.random.random() >= 0.5
            winner = play_eval(reverted, pipe, process_id)
            
            try:
                winner_buffer.put_nowait(winner)
            except:
                break
    
    else:
        while True:
            train_steps = simulate_game(MCTS_steps, MCTS_alpha, pipe, ask_predict, process_id)    
        
            try:
                for step in train_steps:
                    history_buffer.put_nowait(step)
            except:
                break
                    
def stop_simulation():
    global workers
    global history_buffer
    global processable_buffer
    global winner_buffer
    global pipes
    global child_pipes
    
    if 'workers' in globals() and len(workers) != 0:
        for worker in workers:
            worker.terminate()
    workers = []
    
    for pipe in pipes:
        pipe.close()

    for pipe in child_pipes:
        pipe.close()
    
    #for _ in range(history_buffer.qsize()):
    #    try:
    #        history_buffer.get_nowait()
    #    except:
    #        break
            
    #for _ in range(processable_buffer.qsize()):
    #    try:
    #        processable_buffer.get_nowait()
    #    except:
    #        break
        
    history_buffer.close()
    processable_buffer.close()
    winner_buffer.close()
    
    # Then we empty the queues
    del history_buffer
    del processable_buffer
    del pipes
    del child_pipes
    del winner_buffer

def parallel_sim(evaluation=False):
    global workers
    global history_buffer
    global processable_buffer
    global winner_buffer
    global pipes
    global child_pipes
    
    if 'workers' in globals() and len(workers) != 0:
        stop_simulation()
    
    history_buffer = Queue(n_games) # This numbers can be tweaked
    winner_buffer = Queue(eval_games)
    processable_buffer = Queue(num_threads)
    pipes = []
    child_pipes = []
    
    workers = []
    for i in range(num_threads):
        parent_pipe, child_pipe = Pipe() # Pipe to communicate with childs
        pipes.append(parent_pipe)
        child_pipes.append(child_pipe)
        
        worker = Thread(target=sim, args=[i, child_pipe, evaluation])
        worker.daemon = False
        worker.start()
        workers.append(worker)

In [16]:
import tensorflow as tf

def softmax_cross_entropy_with_logits(y_true, y_pred):

    p = y_pred
    pi = y_true

    zero = tf.zeros(shape = tf.shape(pi), dtype=tf.float32)
    where = tf.equal(pi, zero)

    negatives = tf.fill(tf.shape(pi), -100.0) 
    p = tf.where(where, negatives, p)

    loss = tf.maximum(0., tf.nn.softmax_cross_entropy_with_logits(labels = pi, logits = p) - 1)

    return loss

def categorical_weighted(y_true, y_pred):
    return tf.maximum(0., keras.losses.categorical_crossentropy(y_true, y_pred) - 0.5)

In [17]:
alphabot, _ = create_model()
alphabot.compile(optimizer=Adam(1e-4), # SGD(1e-3, momentum=0.9)
                          loss={'value' : 'mse', 'policy': categorical_weighted},
                          loss_weights={'value' : 0.5, 'policy' : 0.5})
alphabot.summary(line_length=112)

________________________________________________________________________________________________________________
Layer (type)                        Output Shape             Param #       Connected to                         
input_1 (InputLayer)                (None, None, None, 5)    0                                                  
________________________________________________________________________________________________________________
conv (Conv2D)                       (None, None, None, 64)   2880          input_1[0][0]                        
________________________________________________________________________________________________________________
conv_bn (BatchNormalization)        (None, None, None, 64)   256           conv[0][0]                           
________________________________________________________________________________________________________________
conv_lkrelu (LeakyReLU)             (None, None, None, 64)   0             conv_bn[0][0]        

In [18]:
#alphabot.save('alphabot_best.pickle')
alphabot_best = load_model('alphabot_best.pickle', 
                           custom_objects={'categorical_weighted' : categorical_weighted, 'tf': tf})

In [21]:
reload_best()

In [224]:
load_best('alphabot_best.pickle')

In [23]:
# History of games for training
complete_history = []

# Game Params
n_players = 2
n_games = 8_000 #10_000 # Simulate N games before each training
k = 5 # Games to be stored n_games * K

# Eval options
allow_move = False
use_eval_choice = False

# Simulation Params
num_threads = 6

MCTS_steps = 70
MCTS_eval_steps = 40
MCTS_eval_steps2 = MCTS_eval_steps
MCTS_alpha = 1.
MCTS_eval_alpha = 1.

# Training Params
t_steps = 2000 # Steps of training
eval_steps = 500 # How many steps before evaluation
eval_games = 200 # How many games to play to evaluate who's best model
win_percent = 0.55 # Ratio of game won to become best model
BATCH_SIZE = 512
total_improv = 0

In [24]:
#complete_history = []
cycles = 1000

train(cycles)

KeyboardInterrupt: 

In [27]:
K.set_value(alphabot.optimizer.lr, 1e-3)

In [40]:
game = emulator.Game(2)
mapp = game.reset()

tree = MCTS()
tree.alpha = 1

old_mapp = None
turn = 0
s = map_to_state(mapp, old_mapp, None, 0)
old_mapp = copy.deepcopy(mapp)
head = None

In [61]:
for i in range(10_000):
    picked_data = random.sample(complete_history, k=min(BATCH_SIZE, len(complete_history)))
    #picked_data = complete_history[0:50]
    picked_data = apply_simmetries(picked_data)
    
    state = []
    policy = []
    value = []
    for step in picked_data:
        policy.append(step.policy)
        state.append(step.state)
        value.append(step.value)

    y = [np.zeros((len(state), 4)), np.zeros((len(state), 1))]
    y[0] = policy
    y[1] = value
    
    losses = alphabot.train_on_batch(np.array(state, dtype=np.float32), y)
    print(losses)

[0.8292602, 0.3044496, 0.98979956]
[0.8159536, 0.29261062, 0.9810097]
[0.80330974, 0.27280754, 0.9854906]
[0.79731065, 0.27124688, 0.9758969]
[0.80080616, 0.27958468, 0.97044516]
[0.7920474, 0.24840118, 0.99976754]
[0.83949023, 0.3147953, 0.9951276]
[0.8497128, 0.3317, 0.990282]
[0.84304434, 0.31689027, 0.9992264]
[0.8196093, 0.2841468, 1.0015377]
[0.8135063, 0.2901465, 0.9803965]
[0.78683156, 0.25361547, 0.98190707]
[0.79597783, 0.27031085, 0.97521883]
[0.82310975, 0.29202205, 0.9969781]
[0.822308, 0.29671633, 0.988396]
[0.8125647, 0.28423315, 0.9876971]
[0.82579094, 0.28991738, 1.0056865]
[0.82539934, 0.30539665, 0.9817473]
[0.78835404, 0.26108143, 0.9741919]
[0.8077688, 0.27207044, 0.9966002]
[0.79170793, 0.2560571, 0.98855996]
[0.7959459, 0.26164576, 0.988714]
[0.7756249, 0.24004437, 0.9805358]
[0.8076964, 0.27125594, 0.99792296]
[0.78949535, 0.2605843, 0.97758955]
[0.7810649, 0.24758412, 0.98029137]
[0.8175474, 0.28866205, 0.99170226]
[0.7787319, 0.2460186, 0.9781015]
[0.79473454,

KeyboardInterrupt: 

In [70]:
step = complete_history[58]
state = step.state
value = step.value
pol = step.policy
policy, v = alphabot.predict(state[np.newaxis])
policy = policy[0]

mapp = np.full((5, 5), -1)
mapp[state[..., 0] == 1] = 1
mapp[state[..., 2] == 1] = 1

valid_actions = emulator.Game(2).valid_actions(mapp, state, state[..., -1].all() == 1)
if len(valid_actions) < 4:
    missing_idx = [v for v in [0, 1, 2, 3] if v not in valid_actions]
    policy[missing_idx] = 0
             
if sum(policy) > 0:
    policy = policy / sum(policy)


policy, pol, value, v

(array([0.40908018, 0.        , 0.59091985, 0.        ], dtype=float32),
 array([0.05668016, 0.        , 0.94331984, 0.        ]),
 1,
 array([[0.4654884]], dtype=float32))

In [229]:
for i in range(1):
    states = simulate_game(10, 0.8, alphabot=alphabot, eval_g=True, return_state=True)

maps = []
for state in states:
    mapp = state[..., 0]
    mapp += state[..., 2] * 2
    mapp[np.where(state[..., 1] == 1)] = 3
    mapp[np.where(state[..., 3] == 1)] = 4
    mapp = np.expand_dims(mapp, axis=-1)
    mapp = np.tile(mapp, [1, 1, 3])
        
    idx, cols, c = np.where(mapp == 1)
    mapp[idx, cols, :] = 0
    mapp[idx, cols, 0] = 128
    
    idx, cols, c = np.where(mapp == 2)
    mapp[idx, cols, :] = 0
    mapp[idx, cols, 1] = 128
    
    idx, cols, c = np.where(mapp == 3)
    mapp[idx, cols, :] = 0
    mapp[idx, cols, 0] = 255
    
    idx, cols, c = np.where(mapp == 4)
    mapp[idx, cols, :] = 0
    mapp[idx, cols, 1] = 255
    
    mapp = cv2.resize(mapp.astype(np.uint8), (480, 480), interpolation=cv2.INTER_AREA)
    
    maps.append(mapp)
    
maps = np.array(maps)

#write_gif(maps, './test.gif', fps=5)

In [32]:
logging.getLogger().setLevel(logging.INFO)

In [78]:
%load_ext autoreload
%autoreload 2

In [20]:
# History of games for training
#complete_history = []

# Game Params
n_players = 2
n_games = 1 #8_000 #10_000 # Simulate N games before each training
k = 5 # Games to be stored n_games * K

# Eval options
allow_move = False
use_eval_choice = False

# Simulation Params
num_threads = 6

MCTS_steps = 70
MCTS_eval_steps = 40
MCTS_eval_steps2 = MCTS_eval_steps
MCTS_alpha = 1.
MCTS_eval_alpha = 1.

# Training Params
t_steps = 2000 # Steps of training
eval_steps = 1 #500 # How many steps before evaluation
eval_games = 200 # How many games to play to evaluate who's best model
win_percent = 0.55 # Ratio of game won to become best model
BATCH_SIZE = 512
total_improv = 0

cycles = 1000

train(cycles)

Process Process-16:
Process Process-13:
Process Process-18:
Process Process-14:
Process Process-17:
Process Process-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootst

  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "src/mcts.py", line 32, in search
    raw_prediction = pipe.recv()
KeyboardInterrupt
  File "src/mcts.py", line 89, in search
    v = self.search(sp, new_map, game, pipe, ask_predict, process_id, allow_move, alphabot, head_pos)
  File "src/mcts.py", line 89, in search
    v = self.search(sp, new_map, game, pipe, ask_predict, process_id, allow_move, alphabot, head_pos)
  [Previous line repeated 2 more times]
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "src/mcts.py", line 32, in search
    raw_prediction = pipe.recv()
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/adryw/miniconda3/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
Ke

KeyboardInterrupt: 

In [320]:
complete_history[506].state[..., 2], complete_history[503].policy

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([0.        , 0.14814815, 0.33333333, 0.51851852]))

In [53]:
np.array([s.state for s in complete_history[506:1000:4]])[:, :, ::-1][0, ..., 2]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])